# 데이터분석5: WebCrawling

## Ⅰ 셀레니움 연습

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By


driver = webdriver.Chrome()

driver.get('https://selenium.dev/documentation')
assert 'Selenium' in driver.title

elem = driver.find_element(By.ID, 'm-documentationwebdriver')
elem.click()
assert 'WebDriver' in driver.title

driver.quit()

In [3]:
# import the required library
from selenium import webdriver
 
# initialize an instance of the chrome driver (browser)
driver = webdriver.Chrome()

# visit your target site
driver.get("https://www.scrapingcourse.com/ecommerce/")

# output the full-page HTML
print(driver.page_source)

# release the resources allocated by Selenium and shut down the browser
driver.quit()

<html lang="en-US"><head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="profile" href="https://gmpg.org/xfn/11">
<link rel="pingback" href="https://www.scrapingcourse.com/ecommerce/xmlrpc.php">
<!-- Google tag (gtag.js) -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-NZGD14H87G"></script>
<script>
	window.dataLayer = window.dataLayer || [];
	function gtag(){dataLayer.push(arguments);}
	gtag('js', new Date());
	gtag('config', 'G-NZGD14H87G');
</script>
<title>Ecommerce Test Site to Learn Web Scraping – ScrapingCourse.com</title>
<meta name="robots" content="max-image-preview:large">
<link rel="dns-prefetch" href="//stats.wp.com">
<link rel="dns-prefetch" href="//www.scrapingcourse.com">
<link rel="dns-prefetch" href="//fonts.googleapis.com">
<link rel="alternate" type="application/rss+xml" title="Ecommerce Test Site to Learn Web Scraping » Feed" href="https://www.scrapingcourse.com/ecommerce/feed/">
<link r

## ⅠⅠ 셀레니움 실습1: qoo10 상품 정보 수집

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

In [5]:
driver = webdriver.Chrome()
driver.get("https://www.qoo10.jp/s/VT-COSMETICS?keyword=vt+cosmetics&keyword_auto_change=&furusato_gdlc_cd=")

items = driver.find_elements(By.XPATH, "//*[@id='catalog_result_item_list']/li/a/div[2]/span[2]")

product_names = []
for item in items:
    product_names.append(item.text)

df = pd.DataFrame({'CICAマスククレンザー , 170ml': product_names})
df.to_csv('vtcosmetics_products.csv', index=False, encoding='utf-8-sig')

driver.quit()

print(df)
# Output the DataFrame to a CSV file

          CICAマスククレンザー , 170ml
0          CICAマスククレンザー, 170ml
1        CICA スポットパッチ, 48枚, 2個
2   CICA デイリースージングマスク, 30枚, 2個
3          シカレチA セブンデイズマスク, 7枚
4    CICA マイルドフォームクレンザー, 300ml
5            リードルショット100, 50ml
6                             
7                             
8                             
9                             
10                            
11                            
12                            
13                            
14                            
15                            
16                            
17                            
18                            
19                            
20                            
21                            
22                            
23                            
24                            
25                            
26                            
27                            
28                            
29                            


## ⅠⅠ 셀레니움 실습2: 왓챠피디아 영화 정보 수집

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import pandas as pd

# 1. 셀레니움 셋업
options = Options()
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=options)

try:
    # 2. 메인 페이지 열기
    url = "https://pedia.watcha.com/ko-KR/?domain=movie"
    driver.get(url)
    wait = WebDriverWait(driver, 10)  # 최대 10초 대기
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#root > div.kVmNFwyR > section > div > section > div:nth-child(2) > section > div.dmYmGzMK.listWrapper")))
    # 3. '박스오피스 순위' 섹션 찾기
    boxoffice_section = driver.find_elements(By.CSS_SELECTOR, "#root > div.kVmNFwyR > section > div > section > div:nth-child(2) > section > div.dmYmGzMK.listWrapper")

    # 4. 영화 카드 리스트 가져오기 (최대 10개)
    movie_cards = boxoffice_section[0].find_elements(By.CSS_SELECTOR, "a.css-1m0nb8u")[:10]
    results = []

    for card in movie_cards:
        movie_title = card.get_attribute("aria-label")
        movie_link = card.get_attribute("href")

        # 5. 영화 상세 페이지 접속
        driver.get(movie_link)
        time.sleep(2)  # 로딩 대기

        try:
            # 평점 요소 찾기
            rating_elem = driver.find_elements(By.CSS_SELECTOR, "span.css-1x9j5hq")  # 평점 클래스, 필요시 조정
            rating = rating_elem[0].text if rating_elem else "평점 없음"
        except:
            rating = "평점 없음"

        print(f"{movie_title} - 평점: {rating}")
        results.append([movie_title, rating])

        # 메인 페이지로 다시 이동
        driver.back()
        time.sleep(2)

    # 6. CSV 저장
    with open("볼 영화 찾기.csv", "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["영화 제목", "평점"])
        writer.writerows(results)

finally:
    driver.quit()

# 7. CSV 파일 읽기
df = pd.read_csv("볼 영화 찾기.csv")
print(df)
df 

Empty DataFrame
Columns: [영화 제목, 평점]
Index: []


,영화 제목,평점


### *수집 실패 - 팝업 때문일지도? → 팝업을 닫는 작업을 추가해보자

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import pandas as pd

# 1. 셀레니움 셋업
options = Options()
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=options)

try:
    # 2. 메인 페이지 열기
    url = "https://pedia.watcha.com/ko-KR/?domain=movie"
    driver.get(url)
    
    wait = WebDriverWait(driver, 30)  # WebDriverWait 사용

    # 3. 팝업 닫기 처리
    try:
        # 팝업 닫기 버튼을 찾고 클릭
        close_button = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.close-button-class'))  # 팝업 닫기 버튼의 정확한 CSS 셀렉터로 수정
        )
        close_button.click()
        time.sleep(1)  # 팝업 닫은 후 잠시 대기
    except:
        print("팝업을 찾을 수 없거나 이미 닫혔습니다.")

    # 4. '박스오피스 순위' 섹션 찾기
    boxoffice_section = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#root > div.kVmNFwyR > section > div > section > div:nth-child(2) > section > div.dmYmGzMK.listWrapper"))
    )

    # 5. 영화 카드 리스트 가져오기 (최대 10개)
    movie_cards = boxoffice_section.find_elements(By.CSS_SELECTOR, "a.css-1m0nb8u")[:10]

    results = []

    for card in movie_cards:
        movie_title = card.get_attribute("aria-label")
        movie_link = card.get_attribute("href")

        # 6. 영화 상세 페이지 접속
        driver.get(movie_link)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.css-1x9j5hq")))  # 평점 로딩 대기

        try:
            # 평점 요소 찾기
            rating_elem = driver.find_element(By.CSS_SELECTOR, "span.css-1x9j5hq")
            rating = rating_elem.text
        except:
            rating = "평점 없음"

        print(f"{movie_title} - 평점: {rating}")
        results.append([movie_title, rating])

        # 7. 메인 페이지로 다시 이동
        driver.back()
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#root > div.kVmNFwyR > section > div > section > div:nth-child(2) > section > div.dmYmGzMK.listWrapper")))  # 다시 메인페이지 대기

    # 8. CSV 저장
    with open("볼 영화 찾기.csv", "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["영화 제목", "평점"])
        writer.writerows(results)

finally:
    driver.quit()

# 9. CSV 파일 읽기
df = pd.read_csv("볼 영화 찾기.csv")
print(df)


팝업을 찾을 수 없거나 이미 닫혔습니다.
Empty DataFrame
Columns: [영화 제목, 평점]
Index: []


### *수집 실패2 - 팝업을 닫는 css를 찾지 못했다 → 왓챠피디아에서 수집을 막고자 고의적으로 팝업 배치?? → 한번 request+bs를 해보자

In [8]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [9]:
import requests
from bs4 import BeautifulSoup
import csv

# 1. '왓챠피디아' 링크가 있는 페이지 열기
url = "https://pedia.watcha.com/ko-KR/?domain=movie"
response = requests.get(url)

# 2. 페이지 파싱
soup = BeautifulSoup(response.text, "html.parser")

# 3. '왓챠피디아' 링크 추출
watcha_link = None
for link in soup.find_all('a', href=True):
    if "pedia.watcha.com" in link['href']:
        watcha_link = link['href']
        break  # 첫 번째 '왓챠피디아' 링크만 추출

if watcha_link:
    # 4. '왓챠피디아' 페이지로 요청 보내기
    watcha_response = requests.get(watcha_link)
    watcha_soup = BeautifulSoup(watcha_response.text, "html.parser")

    # 5. 영화 제목과 평점 추출
    movie_titles = watcha_soup.find_all("strong", class_="title")  # 영화 제목을 포함하는 태그
    movie_ratings = watcha_soup.find_all("span", class_="css-1x9j5hq")  # 평점 태그 (필요시 수정)

    # 6. 결과 저장
    results = []
    for title, rating in zip(movie_titles, movie_ratings):
        movie_title = title.text.strip()
        movie_rating = rating.text.strip() if rating else "평점 없음"
        print(f"{movie_title} - 평점: {movie_rating}")
        results.append([movie_title, movie_rating])

    # 7. CSV로 저장
    with open("볼 영화 찾기.csv", "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["영화 제목", "평점"])
        writer.writerows(results)

else:
    print("왓챠피디아 링크를 찾을 수 없습니다.")


왓챠피디아 링크를 찾을 수 없습니다.


In [10]:
import requests
from bs4 import BeautifulSoup
import csv

# 1. '왓챠피디아' 링크가 있는 페이지 열기
url = "https://pedia.watcha.com/ko-KR/?domain=movie"
response = requests.get(url)

# 2. 페이지 파싱
soup = BeautifulSoup(response.text, "html.parser")

# 3. '왓챠피디아' 링크 추출 (href 속성에 'pedia.watcha.com' 포함)
watcha_link = None
for link in soup.find_all('a', href=True):
    href = link['href']
    if "pedia.watcha.com" in href:
        watcha_link = href
        print(f"왓챠피디아 링크: {watcha_link}")  # 디버깅을 위해 출력
        break  # 첫 번째 '왓챠피디아' 링크만 추출

if watcha_link:
    # 4. '왓챠피디아' 페이지로 요청 보내기
    watcha_response = requests.get(watcha_link)
    watcha_soup = BeautifulSoup(watcha_response.text, "html.parser")

    # 5. 영화 제목과 평점 추출
    movie_titles = watcha_soup.find_all("strong", class_="title")  # 영화 제목을 포함하는 태그
    movie_ratings = watcha_soup.find_all("span", class_="css-1x9j5hq")  # 평점 태그 (필요시 수정)

    # 6. 결과 저장
    results = []
    for title, rating in zip(movie_titles, movie_ratings):
        movie_title = title.text.strip()
        movie_rating = rating.text.strip() if rating else "평점 없음"
        print(f"{movie_title} - 평점: {movie_rating}")  # 수정된 부분
        results.append([movie_title, movie_rating])

    # 7. CSV로 저장
    with open("볼 영화 찾기.csv", "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["영화 제목", "평점"])
        writer.writerows(results)

else:
    print("왓챠피디아 링크를 찾을 수 없습니다.")


왓챠피디아 링크를 찾을 수 없습니다.


### *수집 실패3 - 역시 안된다... 왓챠피디아 페이지 내 링크들이 출력이 되는지 확인해보자.

In [11]:
import requests
from bs4 import BeautifulSoup

# 1. '왓챠피디아' 링크가 있는 페이지 열기
url = "https://pedia.watcha.com/ko-KR/?domain=movie"
response = requests.get(url)

# 2. 페이지 파싱
soup = BeautifulSoup(response.text, "html.parser")

# 3. 페이지 내 모든 링크 출력 (디버깅용)
for link in soup.find_all('a', href=True):
    print(link['href'])  # 모든 href 값을 출력하여 링크들을 확인


/ko-KR
/ko-KR
/ko-KR/?domain=movie
/ko-KR/?domain=tv
/ko-KR/?domain=book
/ko-KR/?domain=webtoon
/ko-KR
/ko-KR/?domain=movie
/ko-KR/?domain=tv
/ko-KR/?domain=book
/ko-KR/?domain=webtoon
https://watcha.com/notices/2049
/ko-KR/contents/m5Z37NR
/ko-KR/contents/mWw3Goq
/ko-KR/contents/mObV01z
/ko-KR/contents/mWq1w3N
/ko-KR/contents/mdKpkjk
/ko-KR/contents/m5DP44y
/ko-KR/contents/mWJXQMx
/ko-KR/contents/mW42GlY
/ko-KR/contents/mWw31aG
/ko-KR/contents/m5DP3A7
/ko-KR/upcoming/movie
/ko-KR/contents/mr5ZXlO
/ko-KR/contents/mdEm6KG
/ko-KR/contents/m5X2zM1
/ko-KR/contents/m5Zl0BR
/ko-KR/contents/mdB7Blz
/ko-KR/contents/mdB7PqG
/ko-KR/contents/mOkLQng
/ko-KR/contents/mdEmq2l
/ko-KR/contents/m5GoGRp
/ko-KR/contents/md6lwny
/ko-KR/contents/mO8X4oV
/ko-KR/contents/mOAkxAj
/ko-KR/contents/mWq1w3N
/ko-KR/contents/mOPV9N7
/ko-KR/contents/mY5QaG5
http://team.watcha.com/
mailto:cs@watchapedia.co.kr
mailto:ad-sales@watcha.com
https://an2-ast.amz.wtchn.net/watchapedia_ad_profile_2024_4.pdf
https://watcha.tea